In [ ]:
import pandas as pd 
import gurobipy as gp
prefs=pd.read_excel('data(1).xlsx',header=[0,1,2],sheet_name='Preferences',index_col=0)
pref_values=prefs.values
req=pd.read_excel('data(1).xlsx',sheet_name='Requirements',index_col=0)
req_values=req.values

slist=range(len(req))
jlist=range(len(prefs))
nightlist=range(2,len(req),3)

mod=gp.Model()

X=mod.addVars(len(prefs),len(req),vtype=gp.GRB.BINARY)
M=mod.addVar()
N=mod.addVar()
m=mod.addVar()
n=mod.addVar()

prefscore=gp.quicksum(X[j,s]*pref_values[j,s] for j in jlist for s in slist)
mod.setObjective(prefscore-100*(M-N)-150*(m-n),sense=gp.GRB.MAXIMIZE)

mod.addConstrs(gp.quicksum(X[j,s] for s in slist)<=M for j in jlist)
mod.addConstrs(gp.quicksum(X[j,s] for s in slist)>=N for j in jlist)
mod.addConstrs(gp.quicksum(X[j,s] for s in nightlist)<=m for j in jlist)
mod.addConstrs(gp.quicksum(X[j,s] for s in nightlist)>=n for j in jlist)
mod.addConstrs(gp.quicksum(X[j,s] for j in jlist)==req_values[s,2] for s in slist)
mod.addConstrs(X[j,s]<=pref_values[j,s] for j in jlist for s in slist)
mod.addConstrs(X[j,s]+X[j,s+1]<=1 for j in jlist for s in range(len(req)-1))
mod.addConstrs(X[j,s-2]+X[j,s]<=1 for j in jlist for s in nightlist)
mod.addConstrs(X[j,s+2]+X[j,s]<=1 for j in jlist for s in nightlist[:-1])
mod.addConstrs(gp.quicksum(X[j,s+i] for i in range(21))<=6 for j in jlist for s in range(0,len(req),21))

mod.optimize()

schedule=prefs.copy()
for i,j in X:
    schedule.iloc[i,j]=X[i,j].X

summary=pd.DataFrame({'Value':[mod.objVal,prefscore.getValue(),M.X-N.X,m.X-n.X]},index=['Objective','Total preference score','Shift inequality','Night inequality'])

with pd.ExcelWriter('output_for_data.xlsx') as writer:
    schedule.to_excel(writer, sheet_name='Schedule')
    summary.to_excel(writer, sheet_name='Summary', index=False)

In [ ]:
mod=Model()

X=mod.addVars(n)
sigma=mod.addVars(n)
z=mod.addVars(n.vtype=GRB.BINARY)

mod.setObjective(R[i]*X[i])
mod.addConstr(sum(X)=1)
mod.addConstr(quicksum(X[i]*X[j] for i in range(n) for j in range(n))<=theta)
mod.addConstrs(X[i]-w[i]<=sigma[i] for i in range(n))
mod.addConstrs(-(X[i]-w[i])<=sigma[i] for i in range(n))
mod.addConstr(1/2*sum(sigma)<=delta)
mod.addConstr(eta<=delta)